In [2]:
# Install required packages
!pip install -q chromadb langchain langchain_openai langchain_community python-dotenv requests

## Setup Environment

In [3]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Set OpenRouter API key from environment variables
openrouter_api_key = os.getenv("OPENROUTER_API_KEY")
openrouter_model = os.getenv("OPENROUTER_MODEL", "openai/gpt-3.5-turbo")

print(f"Using OpenRouter model: {openrouter_model}")
print(f"OpenRouter API key loaded: {'Yes' if openrouter_api_key else 'No'}")

Using OpenRouter model: openai/gpt-3.5-turbo
OpenRouter API key loaded: Yes


## Import Libraries

In [4]:
# Import the latest LangChain packages
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.language_models import LLM
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import requests
from typing import Any, List, Mapping, Optional

## Create Custom OpenRouter LLM Class

In [5]:
class OpenRouterLLM(LLM):
    """Custom LLM class for OpenRouter API"""
    
    @property
    def _llm_type(self) -> str:
        return "openrouter"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        url = "https://openrouter.ai/api/v1/chat/completions"
        headers = {
            "Authorization": f"Bearer {openrouter_api_key}",
            "Content-Type": "application/json",
            "HTTP-Referer": "https://localhost",  # Required by OpenRouter
            "X-Title": "ChromaDB Demo"  # Optional, but good practice
        }
        data = {
            "model": openrouter_model,
            "messages": [{"role": "user", "content": prompt}],
            "max_tokens": 2048,
            "temperature": 0.7
        }
        response = requests.post(url, headers=headers, json=data)
        response_json = response.json()
        return response_json['choices'][0]['message']['content']

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"model": openrouter_model}

# Create an instance of the OpenRouter LLM
llm = OpenRouterLLM()

## Create Sample Data

In [6]:
# Create a data directory if it doesn't exist
data_dir = os.path.join(os.getcwd(), "data")
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"Created directory: {data_dir}")
else:
    print(f"Directory already exists: {data_dir}")

# Create a sample text file for testing
sample_file_path = os.path.join(data_dir, "sample.txt")
if not os.path.exists(sample_file_path):
    with open(sample_file_path, "w") as f:
        f.write("This is a sample document for testing ChromaDB with OpenRouter.\n")
        f.write("ChromaDB is a vector database that can be used for semantic search.\n")
        f.write("OpenRouter provides access to various LLM models through a unified API.\n")
    print(f"Created sample file: {sample_file_path}")
else:
    print(f"Sample file already exists: {sample_file_path}")

# Create another sample file
ai_file_path = os.path.join(data_dir, "ai_models.txt")
if not os.path.exists(ai_file_path):
    with open(ai_file_path, "w") as f:
        f.write("Large Language Models (LLMs) are transforming how we interact with AI.\n")
        f.write("Models like GPT-3.5, GPT-4, Claude, and Llama are available through OpenRouter.\n")
        f.write("Vector databases help store and retrieve embeddings for semantic search applications.\n")
    print(f"Created AI models file: {ai_file_path}")
else:
    print(f"AI models file already exists: {ai_file_path}")

Directory already exists: d:\LLM-Ops\Tutorials\Vector-Databases\data
Sample file already exists: d:\LLM-Ops\Tutorials\Vector-Databases\data\sample.txt
AI models file already exists: d:\LLM-Ops\Tutorials\Vector-Databases\data\ai_models.txt


## Load Documents

In [7]:
# Set up the document loader to use the local data directory
loader = DirectoryLoader(data_dir, glob="**/*.txt", loader_cls=TextLoader)
documents = loader.load()

# Display the loaded documents
print(f"Loaded {len(documents)} documents:")
for doc in documents:
    print(f"\nDocument: {doc.metadata['source']}")
    print(f"Content: {doc.page_content[:100]}...")

Loaded 2 documents:

Document: d:\LLM-Ops\Tutorials\Vector-Databases\data\ai_models.txt
Content: Large Language Models (LLMs) are transforming how we interact with AI.
Models like GPT-3.5, GPT-4, C...

Document: d:\LLM-Ops\Tutorials\Vector-Databases\data\sample.txt
Content: This is a sample document for testing ChromaDB with OpenRouter.
ChromaDB is a vector database that c...


## Create Embeddings with OpenRouter

In [8]:
# Create an instance of OpenAI embeddings using OpenRouter
try:
    # First, test if the embeddings work by embedding a simple string
    embeddings = OpenAIEmbeddings(
        openai_api_base="https://openrouter.ai/api/v1",
        openai_api_key=openrouter_api_key,
        model="text-embedding-ada-002",  # Note: removed "openai/" prefix which might cause issues
        headers={
            "HTTP-Referer": "https://localhost",  # Required by OpenRouter
            "X-Title": "ChromaDB Demo"  # Optional, but good practice
        }
    )
    
    # Test the embeddings with a simple string
    test_embedding = embeddings.embed_query("This is a test")
    print(f"✅ Embeddings test successful! Vector length: {len(test_embedding)}")
    
except Exception as e:
    print(f"❌ Error creating embeddings: {str(e)}")
    print("\nTroubleshooting tips:")
    print("1. Check if your OpenRouter API key is valid")
    print("2. Verify that OpenRouter supports the embedding model you're trying to use")
    print("3. Check your internet connection")
    print("4. Try using a different embedding model")
    
    # Fallback to using HuggingFace embeddings if OpenRouter fails
    print("\nFalling back to HuggingFace embeddings...")
    from langchain_community.embeddings import HuggingFaceEmbeddings
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

❌ Error creating embeddings: Error code: 404 - {'error': {'message': 'Not Found', 'code': 404}}

Troubleshooting tips:
1. Check if your OpenRouter API key is valid
2. Verify that OpenRouter supports the embedding model you're trying to use
3. Check your internet connection
4. Try using a different embedding model

Falling back to HuggingFace embeddings...


C:\Windows\Temp\ipykernel_3336\2746843392.py:29: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


## Create ChromaDB Vector Store

In [9]:
# Create a Chroma vector store
try:
    vector_store = Chroma.from_documents(documents, embeddings)
    print("ChromaDB vector store created successfully!")
except Exception as e:
    print(f"❌ Error creating vector store: {str(e)}")
    print("\nTroubleshooting tips:")
    print("1. Check if your embeddings are working correctly")
    print("2. Verify that your documents are properly loaded")
    print("3. Try using a different embedding model")

ChromaDB vector store created successfully!


## Test Similarity Search

In [10]:
# Test similarity search
try:
    query = "What is ChromaDB used for?"
    results = vector_store.similarity_search(query, k=2)
    
    print(f"Query: {query}")
    print("\nResults:")
    for i, doc in enumerate(results):
        print(f"\nResult {i+1}:")
        print(f"Source: {doc.metadata['source']}")
        print(f"Content: {doc.page_content}")
except Exception as e:
    print(f"❌ Error performing similarity search: {str(e)}")

Query: What is ChromaDB used for?

Results:

Result 1:
Source: d:\LLM-Ops\Tutorials\Vector-Databases\data\sample.txt
Content: This is a sample document for testing ChromaDB with OpenRouter.
ChromaDB is a vector database that can be used for semantic search.
OpenRouter provides access to various LLM models through a unified API.


Result 2:
Source: d:\LLM-Ops\Tutorials\Vector-Databases\data\ai_models.txt
Content: Large Language Models (LLMs) are transforming how we interact with AI.
Models like GPT-3.5, GPT-4, Claude, and Llama are available through OpenRouter.
Vector databases help store and retrieve embeddings for semantic search applications.



## Create a Question-Answering Chain

In [11]:
# Create a prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Please respond to the user request only based on the given context"),
    ("user", "Question: {question}\nContext: {context}")
])

# Create the output parser
output_parser = StrOutputParser()

# Create the chain
chain = prompt | llm | output_parser

## Test the QA Chain

In [12]:
# Define a question
try:
    question = "What is OpenRouter and how does it relate to ChromaDB?"
    
    # Retrieve relevant documents
    docs = vector_store.similarity_search(question, k=2)
    context = "\n\n".join([doc.page_content for doc in docs])
    
    # Run the chain
    response = chain.invoke({"question": question, "context": context})
    
    print(f"Question: {question}")
    print(f"\nAnswer: {response}")
except Exception as e:
    print(f"❌ Error running QA chain: {str(e)}")

Question: What is OpenRouter and how does it relate to ChromaDB?

Answer: OpenRouter provides access to various Large Language Models (LLMs) like GPT-3.5, GPT-4, Claude, and Llama through a unified API. ChromaDB is a vector database that can be used for semantic search applications.


## Try Another Question

In [13]:
# Define another question
try:
    question = "What LLM models are available through OpenRouter?"
    
    # Retrieve relevant documents
    docs = vector_store.similarity_search(question, k=2)
    context = "\n\n".join([doc.page_content for doc in docs])
    
    # Run the chain
    response = chain.invoke({"question": question, "context": context})
    
    print(f"Question: {question}")
    print(f"\nAnswer: {response}")
except Exception as e:
    print(f"❌ Error running QA chain: {str(e)}")

Question: What LLM models are available through OpenRouter?

Answer: The LLM models available through OpenRouter include GPT-3.5, GPT-4, Claude, and Llama.
